In [1]:
# Install required packages and set up Chrome WebDriver
!pip install selenium beautifulsoup4
!apt-get update
!apt-get -q -y install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver_manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [860 kB]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [983 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates

In [2]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def scrape_reviews(url, driver):
    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        return None

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    return review_elements

In [4]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # To run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

# Initialize empty lists to store the data
names = []
comments = []
dates = []

page = 1
stop = False  # Initialize the 'stop' variable

while not stop:
    url = f"https://www.pistonheads.com/gassing/topic.asp?h=0&f=23&t=2023977&i={page * 20}"

    driver.get(url)
    time.sleep(2)

    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@class, 'btn-group-btn') and contains(@class, 'next')]"))
        )
        next_button.click()
    except (TimeoutException, NoSuchElementException, StaleElementReferenceException):
        stop = True  # Set 'stop' to True to exit the loop
        break

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_elements = soup.find_all("div", class_=lambda x: x and x.startswith("topic-reply"))

    if len(review_elements) == 0:
        stop = True
        break

    for review in review_elements:
        name_element = review.find("p", class_="author js-author")
        comment_element = review.find("div", class_="phml msg-body")
        date_element = review.find("span", class_="timestamp")

        name = name_element.text.strip() if name_element else ""
        comment = comment_element.text.strip() if comment_element else ""
        date = date_element.text.strip() if date_element else ""

        names.append(name)
        comments.append(comment)
        dates.append(date)

    page += 1

In [5]:
# Close the webdriver
driver.quit()

In [6]:
# Create a pandas DataFrame from the scraped data
pistonheads3 = pd.DataFrame({
    "Name": names,
    "Comment": comments,
    "Date": dates
})

In [7]:
# Display the review data
print(pistonheads3)

                 Name                                            Comment  \
0         fantheman80  No one mentioned the speaker that plays out br...   
1        Julian Scott  JJJ. said:\n\nspikyone said:\n Thirty-fking-ei...   
2        Julian Scott  Big Rat said:\n So I bought this for Mrs Rat r...   
3        Julian Scott  dpop said:\n 39 grand for something that looks...   
4         Water Fairy  I actually don't mind the look of this having ...   
..                ...                                                ...   
101           nismo48  ducnick said:\n Wow. Just wow. Never have I wa...   
102            biggbn  Mr Tidy said:\n But that only works if you can...   
103  loudlashadjuster  biggbn said:\n\nMr Tidy said:\n But that only ...   
104        garypotter  MOBB said:\n Having had a 595 Abarth before, a...   
105       DonkeyApple  loudlashadjuster said:\n\nbiggbn said:\n\nMr T...   

                    Date  
0    Wednesday 1st March  
1    Wednesday 1st March  
2    W

In [9]:
# Save the DataFrame to a CSV file
csv_filename = "pistonheads3.csv"  # You can choose the desired file name
pistonheads3.to_csv(csv_filename, index=False)

In [10]:
# Display a message after saving
print(f"Scraped data saved to {csv_filename}")

Scraped data saved to pistonheads3.csv


In [11]:
import os

In [12]:
# Define the folder name
folder_name = "Colab_Data"

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Create the folder in Google Drive
drive_path = "/content/drive/MyDrive/"
folder_path = os.path.join(drive_path, folder_name)

In [15]:
# Create the folder if it doesn't exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [16]:
csv_filename = os.path.join(folder_path, "pistonheads3.csv")
pistonheads3.to_csv(csv_filename, index=False)